# Pathway enrichment in MAFA WT T0
Collab Theis - Hebrok labs

Data analysis: Alexander Fastner

based on Data analysis by: Sara Jimenez

Data generation: Veronica Cochrane

## Loading Packages

In [ ]:
import scanpy as sc
import decoupler as dc

# Only needed for processing
import numpy as np
import pandas as pd
from anndata import AnnData
from scipy.stats import pearsonr, spearmanr


## Load Data

In [ ]:
# Read raw data and process it
adata = pd.read_csv('../data/MAFA_WT_gene_count.txt', index_col=20, sep='\t').T
adata

In [ ]:
# Transform to AnnData object
adata = AnnData(adata, dtype=np.float32)
adata.var_names_make_unique()
adata

In [ ]:
# Read metadata 
metadata = pd.read_csv('../data/2023_MAFA_MetaData.txt', index_col=0, sep='\t')
metadata

In [ ]:
adata.obs['condition'] = metadata['Treatment']
adata.obs['time_point'] = metadata['TimePoint']
adata.obs['DOX'] = metadata['DOX']
adata.obs

In [ ]:
# split data set by timepoint
subset_t0 = adata[adata.obs.time_point == 'T0']
print("Subset for time point T0:")
print(subset_t0)
subset_t0.obs
# print("\nSubset for time point T1:")
# print(subset_t1)
# print("\nSubset for time point T2:")
# print(subset_t2)

## Quality Control

In [ ]:
#check various filter points
#old
dc.plot_filter_by_expr(adata, group=None, min_count=10, min_total_count=15, large_n=1, min_prop=1)
dc.plot_filter_by_expr(subset_t0, group=None, min_count=10, min_total_count=15, large_n=1, min_prop=1,save='../figures/WT/t0_QC.png')


In [ ]:
# Obtain genes that pass the thresholds
genes = dc.filter_by_expr(subset_t0, group=None, min_count=10, min_total_count=15, large_n=1, min_prop=1)
genes.size

In [ ]:
# Filter by these genes
adata = subset_t0[:, genes].copy()
adata

In [ ]:
#!pip install pydeseq2

In [ ]:
# Import DESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [ ]:
adata.X = np.round(adata.X,0)

In [ ]:
# Build DESeq2 object
dds = DeseqDataSet(
    adata=adata,
    design_factors='condition',
    refit_cooks=True,
    n_cpus=8,
)

In [ ]:
# Compute LFCs
dds.deseq2()

In [ ]:
# Extract contrast 
stat_res = DeseqStats(dds, contrast=["condition", 'MAFA', 'Control'], n_cpus=8)

In [ ]:
# Compute Wald test
stat_res.summary()

In [ ]:
# Shrink LFCs
stat_res.lfc_shrink(coeff='condition_MAFA_vs_Control')

In [ ]:
# Extract results
results_df = stat_res.results_df
results_df

In [ ]:
#!pip install adjustText

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 6))
dc.plot_volcano_df(
    results_df,
    x='log2FoldChange',
    y='padj',
    top=20,
    ax=ax
)
ax.set_xlim(-10, 10)
ax.set_ylim(0, 300)
plt.savefig('../figures/WT/t0_volcano.png')

In [ ]:
mat = results_df[['stat']].T.rename(index={'stat': 'treatment.vs.control'})
mat

### Transcription factor activity inference

In [ ]:
collectri = dc.get_collectri(organism='human')
#collectri.to_csv('colllectri.csv', index=False)
collectri

### Activity inference with Univariate Linear Model (ULM)

In [ ]:
# Infer TF activities with ulm
tf_acts, tf_pvals = dc.run_ulm(mat=mat, net=collectri, verbose=True)
tf_acts

In [ ]:
dc.plot_barplot(
    tf_acts,
    'treatment.vs.control',
    top=25,
    vertical=True,
    vmin=4,
    vmax=4,
    save='../figures/WT/t0_tf_activities.png'
)

# TF Activity with MLM

In [ ]:
tf_acts_MLM, tf_pvals_MLM, = dc.run_mlm(mat=mat, net=collectri, verbose=True)
tf_acts_MLM

In [ ]:
dc.plot_barplot(
    tf_acts_MLM,
    'treatment.vs.control',
    top=25,
    vertical=True,
    vmin=4,
    vmax=4,
    save='../figures/WT/t0_tf_activities_MLM.png'
)

# Compare ULM and MLM results correlations

In [ ]:
pearson_r, pearson_p = pearsonr(tf_acts.values.flatten(), tf_acts_MLM.values.flatten())
spearman_r, spearman_p = spearmanr(tf_acts.values.flatten(), tf_acts_MLM.values.flatten())

print(f"Pearson correlation: {pearson_r:.2f} (p-value: {pearson_p:.2e})")
print(f"Spearman correlation: {spearman_r:.2f} (p-value: {spearman_p:.2e})")

# Find overlap in (Top 25)

In [ ]:
ulm_top_25 = tf_acts.T.sort_values(by='treatment.vs.control', ascending=False).head(25).index
mlm_top_25 = tf_acts_MLM.T.sort_values(by='treatment.vs.control', ascending=False).head(25).index
len(set(ulm_top_25)&set(mlm_top_25)) / float(len(set(ulm_top_25) | set(mlm_top_25))) * 100

# Find overlap in top 10% (Top 65)

In [ ]:
ulm_top = tf_acts.T.sort_values(by='treatment.vs.control', ascending=False).head(int(len(tf_acts.T) * 0.1)).index
mlm_top = tf_acts_MLM.T.sort_values(by='treatment.vs.control', ascending=False).head(int(len(tf_acts_MLM.T) * 0.1)).index
#len(set(ulm_top)&set(mlm_top)) #length of overlap of both
#float(len(set(ulm_top))) #lengh of first list
# set(mlm_top))) * 100 scale to %
len(set(ulm_top)&set(mlm_top)) / float(len(set(ulm_top) | set(mlm_top))) * 100

In [ ]:
#activated
# dc.plot_targets(results_df, stat='stat', source_name='HNF1A', net=collectri, top=15,save='../figures/WT/t0_HNF1A_targets.png')
# dc.plot_targets(results_df, stat='stat', source_name='SIX3', net=collectri, top=15,save='../figures/WT/t0_SiX3_targets.png')
# dc.plot_targets(results_df, stat='stat', source_name='SREBF1', net=collectri, top=15,save='../figures/WT/t0_SREBF1_targets.png')
# dc.plot_targets(results_df, stat='stat', source_name='PPARA', net=collectri, top=15,save='../figures/WT/t0_PPARA_targets.png')
# dc.plot_targets(results_df, stat='stat', source_name='MTF1', net=collectri, top=15,save='../figures/WT/t0_MTF1_targets.png')
# dc.plot_targets(results_df, stat='stat', source_name='INSM1', net=collectri, top=15,save='../figures/WT/t0_INSM1_targets.png')
# dc.plot_targets(results_df, stat='stat', source_name='NACC1', net=collectri, top=15,save='../figures/WT/t0_NACC1_targets.png')
# #inhibited
# dc.plot_targets(results_df, stat='stat', source_name='E2F1', net=collectri, top=15,save='../figures/WT/t0_E2F1_targets.png')
# #MAFA
# dc.plot_targets(results_df, stat='stat', source_name='MAFA', net=collectri, top=15,save='../figures/WT/t0_MAFA_targets.png')

## Plot Network

In [ ]:
dc.plot_network(
    net=collectri,
    obs=mat,
    act=tf_acts,
    n_sources=['HNF1A', 'SIX3', 'SREBF1', 'PPARA', 'MTF1', 'INSM1', 'NACC1', 'E2F1', 'MAFA'],
    #n_sources=['HNF1A', 'SIX3', 'SREBF1', 'E2F1'],
    n_targets=10,
    node_size=0.5,
    label_size=8,
    figsize=(10, 10),
    c_pos_w='darkgreen',
    c_neg_w='darkred',
    vcenter=True,
    save = '../figures/WT/t0_plot_networks.png'
)

In [ ]:
# Extract logFCs and pvals
logFCs = results_df[['log2FoldChange']].T.rename(index={'log2FoldChange': 'treatment.vs.control'})
pvals = results_df[['padj']].T.rename(index={'padj': 'treatment.vs.control'})

# Plot
dc.plot_volcano(logFCs, pvals, 'treatment.vs.control', name='MAFA', net=collectri, top=10, sign_thr=0.05, lFCs_thr=0.5)

### Pathway activity inference

In [ ]:
# Retrieve PROGENy model weights
progeny = dc.get_progeny(top=500)
progeny

### Activity inference with multivariate Linear Model (MLM)

In [ ]:
# Infer pathway activities with mlm
pathway_acts, pathway_pvals = dc.run_mlm(mat=mat, net=progeny, verbose=True)
pathway_acts

In [ ]:
dc.plot_barplot(
    pathway_acts,
    'treatment.vs.control',
    top=25,
    vertical=True,
    vmin=-5,
    vmax=5,
    save='../figures/WT/t0_pathway_activities.png'
)

In [ ]:
# increase activity
# dc.plot_targets(results_df, stat='stat', source_name='p53', net=progeny, top=30,save='../figures/WT/t0_p53_targets.png')
# dc.plot_targets(results_df, stat='stat', source_name='EGFR', net=progeny, top=30, save='../figures/WT/t0_EGFR_targets.png')
# dc.plot_targets(results_df, stat='stat', source_name='Androgen', net=progeny, top=30,save='../figures/WT/t0_Androgen_targets.png')
# # decrease activity
# dc.plot_targets(results_df, stat='stat', source_name='JAK-STAT', net=progeny, top=30, save='../figures/WT/t0_JAK-STAT_targets.png')

# Approach to investigate p53 -> PPI -> mechanism? -> ask -> Table with list of targets for each one of the pathways with statistics 

### Funtional enrichment of Biological Terms

In [ ]:
# The Molecular Signatures Database (MSigDB) is a resource containing a collection of gene sets annotated to different biological processes.
# msigdb = dc.get_resource('MSigDB')
# msigdb

In [ ]:
# msigdb['collection'].unique()

In [ ]:
# # Filter by hallmark
# msigdb = msigdb[msigdb['collection']=='hallmark']

# # Remove duplicated entries
# msigdb = msigdb[~msigdb.duplicated(['geneset', 'genesymbol'])]

# # Rename
# msigdb.loc[:, 'geneset'] = [name.split('HALLMARK_')[1] for name in msigdb['geneset']]

# msigdb

## ORA

In [ ]:
# # Infer enrichment with ora using significant deg
# top_genes = results_df[results_df['padj'] < 0.05]

# # Run ora
# enr_pvals = dc.get_ora_df(
#     df=top_genes,
#     net=msigdb,
#     source='geneset',
#     target='genesymbol'
# )

# enr_pvals.head()

In [ ]:
# dc.plot_dotplot(
#     enr_pvals.sort_values('Combined score', ascending=False).head(15),
#     x='Combined score',
#     y='Term',
#     s='Odds ratio',
#     c='FDR p-value',
#     scale=0.4,
#     figsize=(5, 10),
#     save='../figures/WT/t0_ORA.png'
# )

: 